# Terrain Classification - Compare Window Size

In [ ]:
import pandas as pd
import glob
import matplotlib.pyplot as plt
import sklearn
sklearn.__version__

#### Data Structure

`Dictionary of Window Sizes
-> Dictionary of Feature Vectors
---> Dictionary of Placements
-----> Dataframe of Processed Data`

In [33]:
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate

# Dictionary of classifiers
classifiers = {'Naive Bayes': GaussianNB(),               'k Nearest': KNeighborsClassifier(),
               'Decision Tree': DecisionTreeClassifier(), 'Random Forest': RandomForestClassifier(),
               'AdaBoost': AdaBoostClassifier(),          'Support Vector Machine': SVC()}

ImportError: cannot import name 'cross_validate'

In [24]:
# Initialize data structure of imported data
size_values = [50, 100, 200, 300, 400, 500]
placements = {'Left': None, 'Middle': None, 'Right': None}
vectors = {vector: placements for vector in ['Features', 'FFTs', 'PSDLogs']}
sizes = {size: vectors for size in size_values}

# For each .csv of processed data
for filename in glob.glob('processed_data/windows/*.csv'):
    placement, vector, size = filename.split('/')[-1].split('.')[0].split('_')
    sizes[int(size)][vector][placement] = pd.read_csv(filename)

In [26]:
def compare_sizes(placement, vector, classifier_names, train_frac=0.8):
    # Figure properties
    fig, ax = plt.subplots(figsize=(10, 8))
    ax.set(title='Window Size Comparision for ' + placement + ' ' + vector)
    ax.set(xlabel='Window Size', y_label='Accuracy')
    
    for classifier_name in classifier_names:
        # Update classifier model
        model = classifiers[classifier_name]
        
        # Accuracy for each window size
        accuracies = []
        
        # Testing time for each window size
        testing_times = []
        
        # For each window size
        for size_value in size_values:
            size_data = sizes[size_value][vector][placement]
            size_labels = size_data.pop('Label')
            # Split data into train and test dataset
            #train_data = size_data.sample(frac=train_frac, random_state=0)
            #test_data = size_data.drop(train.index)
            #train_label = train.pop('Label')
            #test_label = test.pop('Label')
            
            # Fit model to train data
            #model.fit(train_data, train_label)

            # Predict test data with trained model
            #predict_label = model.predict(test_data)
    
            # Compute accuracy of each classifier and update array
            #accuracy = accuracy_score(test_label, predict_label)
            #accuracies.append(accuracy)
            
            # Five-fold cross validate data and get metrics
            scores = cross_validate(model, size_data, size_target, scoring=scoring,
...                         cv=5, return_train_score=True)
            print(sorted(scores.keys()))
            
        plt.plot(size_values, accuracies, label=classifier)

SyntaxError: invalid syntax (<ipython-input-26-1379edaa976b>, line 37)